In [1]:
! pip3 install -qU langchain-upstage

In [2]:

%load_ext dotenv
%dotenv
# set UPSTAGE_API_KEY

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [4]:
# Quick hello world
from langchain_upstage import ChatUpstage # chain처럼 pipe라인으로 연결해라

llm = ChatUpstage() # 엔진
llm.invoke("What's the best season to get to Korean?")

AIMessage(content='The best season to learn Korean depends on your personal preferences and goals. However, many people find that the spring and fall seasons are ideal for learning a new language. These seasons tend to be more relaxed and less hectic, which can make it easier to focus on your studies. Additionally, the mild weather can create a more comfortable learning environment. Ultimately, the most important factor is finding a time that works best for you and your schedule.', response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 21, 'total_tokens': 114}, 'model_name': 'solar-1-mini-chat', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5364217b-a4d6-4ae9-891e-a8ce6fcc5b3e-0')

In [5]:
# Chat prompt
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages( # tuple의 list를 주는 형식
    [
        ("system", "You are a helpful assistant."), # 역할을 줌
        ("human", "What is the capital of France?"), # 아 이걸 연속적으로 이렇게 줄 수 있구나 api에서는
        ("ai", "I know of it. It's Paris!!"),
        ("human", "What about Korea?"), # 대화를 통해 맥락을 줌.. 마지막 human이 명령을 내림
    ]
)

# 3. define chain
from langchain_core.output_parsers import StrOutputParser

chain = chat_prompt | llm | StrOutputParser() # AIMessage type의 결과물을 string type으로 바꾸는 chain
chain.invoke({}) # 보통 prompt + 엔진 + parser 3단 구조로 chain을 구성함



"Oh, I'm not sure about that one. I'll have to think... It's Seoul!!"

In [6]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template( # 바로 string input을 주는 형식
    """
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is
"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({}) # 자꾸 틀림 수학 안 배운 문과라

'To find out how many apples the cafeteria has, we need to follow these steps:\n\n1. Start with the initial number of apples, which is 23.\n2. Subtract the number of apples used for lunch, which is 20.\n3. Add the number of apples bought, which is 6.\n\nSo, the calculation would be:\n\n23 (initial apples) - 20 (used for lunch) + 6 (bought) = 29 apples.\n\nTherefore, the cafeteria has 29 apples.'

In [7]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: the answer is 
"""
) # Chain-of-Thought Prompting, 예제를 만들어 주면 답을 찾음
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'29.'

![CoT](figures/cot.webp)

from https://arxiv.org/abs/2201.11903

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"""
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'The cafeteria started with 23 apples. They used 20 for lunch, leaving them with 3 apples. Then they bought 6 more apples, so they now have 3 + 6 = 9 apples. The answer is 9.'

![Zero-Shot COT](figures/zero-cot.webp)

From https://arxiv.org/abs/2205.11916

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?

A: The answer is 11.

Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?

A: Let's think step by step.
"""
) # step by step으로 똑바로 하라고 말만 해도 성능 나아짐.
# 계산 문제에 대해서 어떻게 일이 일어나는지? 혼자 말하다가 논리의 오류를 찾아내는 것 아닐까. 할머니가 아프다고 애걸복걸해도 잘 된다 카더라... 뭘까...
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'Step 1: The cafeteria started with 23 apples.\nStep 2: They used 20 apples to make lunch, so they had 23 - 20 = 3 apples left.\nStep 3: They bought 6 more apples, so they now have 3 + 6 = 9 apples.\n\nThe answer is 9.'

## divide and conquer
![divideandconquer](figures/divideandconquer.png)

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template( # 문제가 복잡하다 -> 작은 문제로 쪼개서 생각하자
    """
    Please provide three questions from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'\n1. What is SOLAR 10.7B, and what are its key features?\n2. How does the depth up-scaling (DUS) method differ from other LLM up-scaling methods?\n3. What is the significance of SOLAR 10.7B-Instruct, and how does it compare to Mixtral-8x7B-Instruct?'

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template( # 일단 키워드를 뽑아보자
    """
    Please extract three keywords from the following text:
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'\n1. SOLAR 10.7B: This is the name of the large language model introduced in the text.\n2. Depth up-scaling (DUS): This is the method used to scale the LLM, which involves depthwise scaling and continued pretraining.\n3. Instruction-following capabilities: This refers to the fine-tuned variant of SOLAR 10.7B, SOLAR 10.7B-Instruct, which has improved abilities to follow instructions.'

In [12]:
from langchain_core.prompts import PromptTemplate # 랭체인에서 prompt를 template이라고 함

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "Depth up-scaling (DUS)":
    
    ---
    We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
    """
)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({})

'What is the method for scaling large language models (LLMs) called that is introduced in the text, which encompasses depthwise scaling and continued pretraining, and does not require complex changes to train and inference efficiently?'

In [13]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template( # {}는 나중에 invoke할 때 채울 수 있음 formatting 변수
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [14]:
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"adjective": "funny", "content": "chickens"})

'Why did the chicken join a band? Because it had the drumsticks!'

In [15]:
chain.invoke({"adjective": "funny", "content": "beef"}) # 랭체인 이해 끝!

'Why did the beef cross the road?\n\nTo get to the other side of the grill!'

In [16]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
    Please provide one question from the following text 
    regarding "{keyword}":
    
    ---
    {text}
    """
)
chain = prompt_template | llm | StrOutputParser()
keyword = "DUS"
text = """
We introduce SOLAR 10.7B, a large language model (LLM) with 10.7 billion parameters, 
    demonstrating superior performance in various natural language processing (NLP) tasks. 
    Inspired by recent efforts to efficiently up-scale LLMs, 
    we present a method for scaling LLMs called depth up-scaling (DUS), 
    which encompasses depthwise scaling and continued pretraining.
    In contrast to other LLM up-scaling methods that use mixture-of-experts, 
    DUS does not require complex changes to train and inference efficiently. 
    We show experimentally that DUS is simple yet effective 
    in scaling up high-performance LLMs from small ones. 
    Building on the DUS model, we additionally present SOLAR 10.7B-Instruct, 
    a variant fine-tuned for instruction-following capabilities, 
    surpassing Mixtral-8x7B-Instruct. 
    SOLAR 10.7B is publicly available under the Apache 2.0 license, 
    promoting broad access and application in the LLM field.
"""
chain.invoke({"keyword": keyword, "text": text})

'What is the method for scaling large language models (LLMs) called, which encompasses depthwise scaling and continued pretraining, and does not require complex changes to train and inference efficiently?'

# Excercise 

Complete this code to create excellent questions from given documents using the following steps:
1. Include Few Shot examples, including CoT.
2. Generate keywords and topics first.
3. Iterate through each keyword and topic to generate questions.
4. Compare the generated questions with zero-shot generated questions.


# References
* https://platform.openai.com/docs/guides/prompt-engineering 
* https://docs.anthropic.com/claude/docs/intro-to-prompting 
* https://smith.langchain.com/hub 